In [ ]:
import polars as pl
import duckdb
import plotly.graph_objs as go
import plotly.express as px
import missingno as msno

In [ ]:
EUROZONE = [
    "Austria",
    "Belgium",
    "Cyprus",
    "Estonia",
    "Finland",
    "France and Monaco",
    "Germany",
    "Greece",
    "Ireland",
    "Italy, San Marino and the Holy See",
    "Latvia",
    "Lithuania",
    "Luxembourg",
    "Malta",
    "Netherlands",
    "Portugal",
    "Slovakia",
    "Slovenia",
    "Spain and Andorra",
]

# Reading Datasets and Initial Exploration

In [ ]:
df = pl.read_csv("../data/GHG_total_by_country.csv")
df.head()

In [ ]:
# Summary of statistics
df.describe()

In [ ]:
# Check for missing values
with pl.Config() as cfg:
    cfg.set_tbl_cols(-1)
    print(df.null_count())

msno.matrix(df.to_pandas(), sparkline=False)

check_nulls = pl.any_horizontal(pl.col(col).is_null() for col in df.columns)
rows_with_nulls = df.filter(check_nulls)
with pl.Config() as cfg:
    cfg.set_tbl_cols(-1)
    print(rows_with_nulls)

# Data Cleaning and Preprocessing